In [1]:
import pandas as pd
import snappy
import os
from itertools import groupby
import pickle
import numpy as np

In [2]:
#option d'affichage pour les df
pd.options.display.max_rows = 999
pd.options.display.max_columns = None

In [3]:
#choix des colonnes pour l'étude
col = ['x__IMISSIONTRAINNUMBER', 'WC_CAR01_LCST_ISYSTEMLIFESIGN', 'WC_CAR03_LCST_ISYSTEMLIFESIGN', 'WC_CAR05_LCST_ISYSTEMLIFESIGN', 'WC_CAR07_LCST_ISYSTEMLIFESIGN', 'WC_CAR01_LCST_IWSUTANKLEVEL',
'WC_CAR03_LCST_IWSUTANKLEVEL', 'WC_CAR05_LCST_IWSUTANKLEVEL', 'WC_CAR07_LCST_IWSUTANKLEVEL', 'WC_CAR01_LCST_IFWTANKCONTENT', 'WC_CAR03_LCST_IFWTANKCONTENT', 'WC_CAR05_LCST_IFWTANKCONTENT',
'WC_CAR07_LCST_IFWTANKCONTENT', 'WC_CAR01_LCST_IWWTANKCONTENT', 'WC_CAR03_LCST_IWWTANKCONTENT', 'WC_CAR05_LCST_IWWTANKCONTENT', 'WC_CAR07_LCST_IWWTANKCONTENT', 'WC_CAR01_LCST_IWCWORKTIME',
'WC_CAR03_LCST_IWCWORKTIME', 'WC_CAR05_LCST_IWCWORKTIME', 'WC_CAR07_LCST_IWCWORKTIME', 'WC_CAR01_LCST_IWCWORKTIMEINCOMSERVICE', 'WC_CAR03_LCST_IWCWORKTIMEINCOMSERVICE',
'WC_CAR05_LCST_IWCWORKTIMEINCOMSERVICE', 'WC_CAR07_LCST_IWCWORKTIMEINCOMSERVICE', 'WC_CAR01_LCST_IDOORCYCLCNT', 'WC_CAR03_LCST_IDOORCYCLCNT', 'WC_CAR05_LCST_IDOORCYCLCNT',
'WC_CAR07_LCST_IDOORCYCLCNT', 'WC_CAR01_LCST_IFLUSHCNT', 'WC_CAR03_LCST_IFLUSHCNT', 'WC_CAR05_LCST_IFLUSHCNT', 'WC_CAR07_LCST_IFLUSHCNT', 'WC_CAR01_LCST_IEJECTORCNT', 
'WC_CAR03_LCST_IEJECTORCNT', 'WC_CAR05_LCST_IEJECTORCNT', 'WC_CAR07_LCST_IEJECTORCNT',  'WC_CAR01_LCST_IWATERTAPCNT', 'WC_CAR03_LCST_IWATERTAPCNT', 'WC_CAR05_LCST_IWATERTAPCNT', 
'WC_CAR07_LCST_IWATERTAPCNT', 'WC_CAR01_LCST_IDELIMINGCNT', 'WC_CAR03_LCST_IDELIMINGCNT', 'WC_CAR05_LCST_IDELIMINGCNT', 'WC_CAR07_LCST_IDELIMINGCNT', 'WC_CAR01_LCST_IDEFROSTCNT', 
'WC_CAR03_LCST_IDEFROSTCNT', 'WC_CAR05_LCST_IDEFROSTCNT', 'WC_CAR07_LCST_IDEFROSTCNT', 'WC_CAR01_LCST_IFWT05CNT', 'WC_CAR03_LCST_IFWT05CNT', 'WC_CAR05_LCST_IFWT05CNT', 'WC_CAR07_LCST_IFWT05CNT',
'WC_CAR01_LCST_IWWT95CNT', 'WC_CAR03_LCST_IWWT95CNT', 'WC_CAR05_LCST_IWWT95CNT', 'WC_CAR07_LCST_IWWT95CNT', 'WC_CAR01_LCST_FWCOUTOFSERVICE', 'WC_CAR03_LCST_FWCOUTOFSERVICE', 
'WC_CAR05_LCST_FWCOUTOFSERVICE', 'WC_CAR07_LCST_FWCOUTOFSERVICE', 'WC_CAR01_LCST_IWCREADY', 'WC_CAR03_LCST_IWCREADY', 'WC_CAR05_LCST_IWCREADY', 'WC_CAR07_LCST_IWCREADY', 
'WC_CAR01_LCST_FFWTEMPTY', 'WC_CAR03_LCST_FFWTEMPTY', 'WC_CAR05_LCST_FFWTEMPTY', 'WC_CAR07_LCST_FFWTEMPTY']


In [4]:
#adresse des données
REP_DATA = 'D:/ESTACA/4A/Projet industriel/Données/Données'

In [5]:
#liste des répertoires de données (contenant les fichiers parquet)
content_list = os.listdir(REP_DATA)

In [6]:
#création d'un dictionnaire pour les noms de rames (clé = rame, valeur = liste des répertoires)
#on ne garde que la premiere partie du nom des fichiers
racine_dict = {}
for racine, group_rep in groupby(content_list, lambda nom: nom.split('_')[0]):
    racine_dict[racine] = list(group_rep)

In [7]:
#création d'un dictionnaire de df en fonction des rames (clé = rame, valeur = df de données)
#lecture de tous les fichiers qui se rapportent à la même rame
#suppression de la dernière ligne car que des NaN dedans
#conversion du code mission en int pour que les 16 chiffres s'affichent
dict_df = {}
for rac, reps in racine_dict.items():
    df_list = []
    for rep in reps:
        df_temp = pd.read_parquet(REP_DATA + '/' + rep + '/TT_IP.parquet')
        df_temp = df_temp.loc[:, col].iloc[:-1]
        df_temp.x__IMISSIONTRAINNUMBER = df_temp.x__IMISSIONTRAINNUMBER.astype(np.int64)
        df_list.append(df_temp)
    dict_df[rac]=pd.concat(df_list, ignore_index=True)

In [8]:
#sauvegarde du dictionnaire de df des rames en pickle
with open('dict_df.pkl', 'wb') as file:      
    pickle.dump(dict_df, file)

In [9]:
#création du dictionnaire de df en fonction des missions de chaque rame (clé : rame, valeur : dictionnaire(clé = mission, valeur = df de données))
#lecture de toutes les missions possible d'une rame
dict_missions = {}
for rame, datas in dict_df.items():
    dict_mission_rame = {}
    missions = datas.x__IMISSIONTRAINNUMBER.unique()
    for mission in missions :
        df_temp = datas.query('x__IMISSIONTRAINNUMBER == @mission') #@ car mission variable externe a df
        dict_mission_rame[mission] = df_temp
    dict_missions[rame]=dict_mission_rame

In [10]:
#sauvegarde du dictionnaire des missions en fonction des rames dans un pickle
with open('dict_rame_missions.pkl', 'wb') as file:      
    pickle.dump(dict_missions, file)

In [11]:
# exemple de résultat : affichage des missions 0 de la rame z5500503
dict_missions['z5500503'][0]

,x__IMISSIONTRAINNUMBER,WC_CAR01_LCST_ISYSTEMLIFESIGN,WC_CAR03_LCST_ISYSTEMLIFESIGN,WC_CAR05_LCST_ISYSTEMLIFESIGN,WC_CAR07_LCST_ISYSTEMLIFESIGN,WC_CAR01_LCST_IWSUTANKLEVEL,WC_CAR03_LCST_IWSUTANKLEVEL,WC_CAR05_LCST_IWSUTANKLEVEL,WC_CAR07_LCST_IWSUTANKLEVEL,WC_CAR01_LCST_IFWTANKCONTENT,WC_CAR03_LCST_IFWTANKCONTENT,WC_CAR05_LCST_IFWTANKCONTENT,WC_CAR07_LCST_IFWTANKCONTENT,WC_CAR01_LCST_IWWTANKCONTENT,WC_CAR03_LCST_IWWTANKCONTENT,WC_CAR05_LCST_IWWTANKCONTENT,WC_CAR07_LCST_IWWTANKCONTENT,WC_CAR01_LCST_IWCWORKTIME,WC_CAR03_LCST_IWCWORKTIME,WC_CAR05_LCST_IWCWORKTIME,WC_CAR07_LCST_IWCWORKTIME,WC_CAR01_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR03_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR05_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR07_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR01_LCST_IDOORCYCLCNT,WC_CAR03_LCST_IDOORCYCLCNT,WC_CAR05_LCST_IDOORCYCLCNT,WC_CAR07_LCST_IDOORCYCLCNT,WC_CAR01_LCST_IFLUSHCNT,WC_CAR03_LCST_IFLUSHCNT,WC_CAR05_LCST_IFLUSHCNT,WC_CAR07_LCST_IFLUSHCNT,WC_CAR01_LCST_IEJECTORCNT,WC_CAR03_LCST_IEJECTORCNT,WC_CAR05_LCST_IEJECTORCNT,WC_CAR07_LCST_IEJECTORCNT,WC_CAR01_LCST_IWATERTAPCNT,WC_CAR03_LCST_IWATERTAPCNT,WC_CAR05_LCST_IWATERTAPCNT,WC_CAR07_LCST_IWATERTAPCNT,WC_CAR01_LCST_IDELIMINGCNT,WC_CAR03_LCST_IDELIMINGCNT,WC_CAR05_LCST_IDELIMINGCNT,WC_CAR07_LCST_IDELIMINGCNT,WC_CAR01_LCST_IDEFROSTCNT,WC_CAR03_LCST_IDEFROSTCNT,WC_CAR05_LCST_IDEFROSTCNT,WC_CAR07_LCST_IDEFROSTCNT,WC_CAR01_LCST_IFWT05CNT,WC_CAR03_LCST_IFWT05CNT,WC_CAR05_LCST_IFWT05CNT,WC_CAR07_LCST_IFWT05CNT,WC_CAR01_LCST_IWWT95CNT,WC_CAR03_LCST_IWWT95CNT,WC_CAR05_LCST_IWWT95CNT,WC_CAR07_LCST_IWWT95CNT,WC_CAR01_LCST_FWCOUTOFSERVICE,WC_CAR03_LCST_FWCOUTOFSERVICE,WC_CAR05_LCST_FWCOUTOFSERVICE,WC_CAR07_LCST_FWCOUTOFSERVICE,WC_CAR01_LCST_IWCREADY,WC_CAR03_LCST_IWCREADY,WC_CAR05_LCST_IWCREADY,WC_CAR07_LCST_IWCREADY,WC_CAR01_LCST_FFWTEMPTY,WC_CAR03_LCST_FFWTEMPTY,WC_CAR05_LCST_FFWTEMPTY,WC_CAR07_LCST_FFWTEMPTY
0,0,16275.0,16265.0,32616.0,30702.0,126.0,6708.0,667.0,0.0,826.0,824.0,824.0,1648.0,0.0,0.0,0.0,0.0,119812097.0,126258725.0,122605227.0,154144680.0,1.922051e+09,2.132791e+09,1.894073e+09,2.480316e+09,1328523.0,1850184.0,1844713.0,2538925.0,14733801.0,8122632.0,8419309.0,17917656.0,5006449.0,4362996.0,5170455.0,13903387.0,728570.0,711564.0,752514.0,980023.0,246.0,252.0,246.0,246.0,80073.0,88536.0,74948.0,107666.0,23319488.0,564369246.0,64817433.0,46700599.0,11503575.0,76976382.0,26830154.0,5993462.0,413.0,413.0,0.0,412.0,0.0,0.0,412.0,0.0,0.0,0.0,0.0,0.0
1,0,16275.0,16265.0,32616.0,30702.0,126.0,6708.0,667.0,0.0,826.0,824.0,824.0,1648.0,0.0,0.0,0.0,0.0,119812097.0,126258725.0,122605227.0,154144680.0,1.922051e+09,2.132791e+09,1.894073e+09,2.480316e+09,1328523.0,1850184.0,1844713.0,2538925.0,14733801.0,8122632.0,8419309.0,17917656.0,5006449.0,4362996.0,5170455.0,13903387.0,728570.0,711564.0,752514.0,980023.0,246.0,252.0,246.0,246.0,80073.0,88536.0,74948.0,107666.0,23319488.0,564369246.0,64817433.0,46700599.0,11503575.0,76976382.0,26830154.0,5993462.0,413.0,413.0,0.0,412.0,0.0,0.0,412.0,0.0,0.0,0.0,0.0,0.0
2,0,16275.0,16265.0,32616.0,30702.0,126.0,6708.0,667.0,0.0,826.0,824.0,824.0,1648.0,0.0,0.0,0.0,0.0,119812097.0,126258725.0,122605227.0,154144680.0,1.922051e+09,2.132791e+09,1.894073e+09,2.480316e+09,1328523.0,1850184.0,1844713.0,2538925.0,14733801.0,8122632.0,8419309.0,17917656.0,5006449.0,4362996.0,5170455.0,13903387.0,728570.0,711564.0,752514.0,980023.0,246.0,252.0,246.0,246.0,80073.0,88536.0,74948.0,107666.0,23319488.0,564369246.0,64817433.0,46700599.0,11503575.0,76976382.0,26830154.0,5993462.0,413.0,413.0,0.0,412.0,0.0,0.0,412.0,0.0,0.0,0.0,0.0,0.0
3,0,16275.0,16265.0,32616.0,30702.0,126.0,6708.0,667.0,0.0,826.0,824.0,824.0,1648.0,0.0,0.0,0.0,0.0,119812097.0,126258725.0,122605227.0,154144680.0,1.922051e+09,2.132791e+09,1.894073e+09,2.480316e+09,1328523.0,1850184.0,1844713.0,2538925.0,14733801.0,8122632.0,8419309.0,17917656.0,5006449.0,4362996.0,5170455.0,13903387.0,728570.0,711564.0,752514.0,980023.0,246.0,252.0,246.0,246.0,80073.0,88536.0,74948.0,107666.0,23319488.0,564369246.0,64817433.

In [12]:
#exemple de résultat : affichage de toutes les données de z5500503
dict_df['z5500503']

,x__IMISSIONTRAINNUMBER,WC_CAR01_LCST_ISYSTEMLIFESIGN,WC_CAR03_LCST_ISYSTEMLIFESIGN,WC_CAR05_LCST_ISYSTEMLIFESIGN,WC_CAR07_LCST_ISYSTEMLIFESIGN,WC_CAR01_LCST_IWSUTANKLEVEL,WC_CAR03_LCST_IWSUTANKLEVEL,WC_CAR05_LCST_IWSUTANKLEVEL,WC_CAR07_LCST_IWSUTANKLEVEL,WC_CAR01_LCST_IFWTANKCONTENT,WC_CAR03_LCST_IFWTANKCONTENT,WC_CAR05_LCST_IFWTANKCONTENT,WC_CAR07_LCST_IFWTANKCONTENT,WC_CAR01_LCST_IWWTANKCONTENT,WC_CAR03_LCST_IWWTANKCONTENT,WC_CAR05_LCST_IWWTANKCONTENT,WC_CAR07_LCST_IWWTANKCONTENT,WC_CAR01_LCST_IWCWORKTIME,WC_CAR03_LCST_IWCWORKTIME,WC_CAR05_LCST_IWCWORKTIME,WC_CAR07_LCST_IWCWORKTIME,WC_CAR01_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR03_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR05_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR07_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR01_LCST_IDOORCYCLCNT,WC_CAR03_LCST_IDOORCYCLCNT,WC_CAR05_LCST_IDOORCYCLCNT,WC_CAR07_LCST_IDOORCYCLCNT,WC_CAR01_LCST_IFLUSHCNT,WC_CAR03_LCST_IFLUSHCNT,WC_CAR05_LCST_IFLUSHCNT,WC_CAR07_LCST_IFLUSHCNT,WC_CAR01_LCST_IEJECTORCNT,WC_CAR03_LCST_IEJECTORCNT,WC_CAR05_LCST_IEJECTORCNT,WC_CAR07_LCST_IEJECTORCNT,WC_CAR01_LCST_IWATERTAPCNT,WC_CAR03_LCST_IWATERTAPCNT,WC_CAR05_LCST_IWATERTAPCNT,WC_CAR07_LCST_IWATERTAPCNT,WC_CAR01_LCST_IDELIMINGCNT,WC_CAR03_LCST_IDELIMINGCNT,WC_CAR05_LCST_IDELIMINGCNT,WC_CAR07_LCST_IDELIMINGCNT,WC_CAR01_LCST_IDEFROSTCNT,WC_CAR03_LCST_IDEFROSTCNT,WC_CAR05_LCST_IDEFROSTCNT,WC_CAR07_LCST_IDEFROSTCNT,WC_CAR01_LCST_IFWT05CNT,WC_CAR03_LCST_IFWT05CNT,WC_CAR05_LCST_IFWT05CNT,WC_CAR07_LCST_IFWT05CNT,WC_CAR01_LCST_IWWT95CNT,WC_CAR03_LCST_IWWT95CNT,WC_CAR05_LCST_IWWT95CNT,WC_CAR07_LCST_IWWT95CNT,WC_CAR01_LCST_FWCOUTOFSERVICE,WC_CAR03_LCST_FWCOUTOFSERVICE,WC_CAR05_LCST_FWCOUTOFSERVICE,WC_CAR07_LCST_FWCOUTOFSERVICE,WC_CAR01_LCST_IWCREADY,WC_CAR03_LCST_IWCREADY,WC_CAR05_LCST_IWCREADY,WC_CAR07_LCST_IWCREADY,WC_CAR01_LCST_FFWTEMPTY,WC_CAR03_LCST_FFWTEMPTY,WC_CAR05_LCST_FFWTEMPTY,WC_CAR07_LCST_FFWTEMPTY
0,0,16275.0,16265.0,32616.0,30702.0,126.0,6708.0,667.0,0.0,826.0,824.0,824.0,1648.0,0.0,0.0,0.0,0.0,119812097.0,126258725.0,122605227.0,154144680.0,1.922051e+09,2.132791e+09,1.894073e+09,2.480316e+09,1328523.0,1850184.0,1844713.0,2538925.0,14733801.0,8122632.0,8419309.0,17917656.0,5006449.0,4362996.0,5170455.0,13903387.0,728570.0,711564.0,752514.0,980023.0,246.0,252.0,246.0,246.0,80073.0,88536.0,74948.0,107666.0,23319488.0,564369246.0,64817433.0,46700599.0,11503575.0,76976382.0,26830154.0,5993462.0,413.0,413.0,0.0,412.0,0.0,0.0,412.0,0.0,0.0,0.0,0.0,0.0
1,0,16275.0,16265.0,32616.0,30702.0,126.0,6708.0,667.0,0.0,826.0,824.0,824.0,1648.0,0.0,0.0,0.0,0.0,119812097.0,126258725.0,122605227.0,154144680.0,1.922051e+09,2.132791e+09,1.894073e+09,2.480316e+09,1328523.0,1850184.0,1844713.0,2538925.0,14733801.0,8122632.0,8419309.0,17917656.0,5006449.0,4362996.0,5170455.0,13903387.0,728570.0,711564.0,752514.0,980023.0,246.0,252.0,246.0,246.0,80073.0,88536.0,74948.0,107666.0,23319488.0,564369246.0,64817433.0,46700599.0,11503575.0,76976382.0,26830154.0,5993462.0,413.0,413.0,0.0,412.0,0.0,0.0,412.0,0.0,0.0,0.0,0.0,0.0
2,0,16275.0,16265.0,32616.0,30702.0,126.0,6708.0,667.0,0.0,826.0,824.0,824.0,1648.0,0.0,0.0,0.0,0.0,119812097.0,126258725.0,122605227.0,154144680.0,1.922051e+09,2.132791e+09,1.894073e+09,2.480316e+09,1328523.0,1850184.0,1844713.0,2538925.0,14733801.0,8122632.0,8419309.0,17917656.0,5006449.0,4362996.0,5170455.0,13903387.0,728570.0,711564.0,752514.0,980023.0,246.0,252.0,246.0,246.0,80073.0,88536.0,74948.0,107666.0,23319488.0,564369246.0,64817433.0,46700599.0,11503575.0,76976382.0,26830154.0,5993462.0,413.0,413.0,0.0,412.0,0.0,0.0,412.0,0.0,0.0,0.0,0.0,0.0
3,0,16275.0,16265.0,32616.0,30702.0,126.0,6708.0,667.0,0.0,826.0,824.0,824.0,1648.0,0.0,0.0,0.0,0.0,119812097.0,126258725.0,122605227.0,154144680.0,1.922051e+09,2.132791e+09,1.894073e+09,2.480316e+09,1328523.0,1850184.0,1844713.0,2538925.0,14733801.0,8122632.0,8419309.0,17917656.0,5006449.0,4362996.0,5170455.0,13903387.0,728570.0,711564.0,752514.0,980023.0,246.0,252.0,246.0,246.0,80073.0,88536.0,74948.0,107666.0,23319488.0,564369246.0,64817433.

In [14]:
dict_df['z5500503'].describe()

,x__IMISSIONTRAINNUMBER,WC_CAR01_LCST_ISYSTEMLIFESIGN,WC_CAR03_LCST_ISYSTEMLIFESIGN,WC_CAR05_LCST_ISYSTEMLIFESIGN,WC_CAR07_LCST_ISYSTEMLIFESIGN,WC_CAR01_LCST_IWSUTANKLEVEL,WC_CAR03_LCST_IWSUTANKLEVEL,WC_CAR05_LCST_IWSUTANKLEVEL,WC_CAR07_LCST_IWSUTANKLEVEL,WC_CAR01_LCST_IFWTANKCONTENT,WC_CAR03_LCST_IFWTANKCONTENT,WC_CAR05_LCST_IFWTANKCONTENT,WC_CAR07_LCST_IFWTANKCONTENT,WC_CAR01_LCST_IWWTANKCONTENT,WC_CAR03_LCST_IWWTANKCONTENT,WC_CAR05_LCST_IWWTANKCONTENT,WC_CAR07_LCST_IWWTANKCONTENT,WC_CAR01_LCST_IWCWORKTIME,WC_CAR03_LCST_IWCWORKTIME,WC_CAR05_LCST_IWCWORKTIME,WC_CAR07_LCST_IWCWORKTIME,WC_CAR01_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR03_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR05_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR07_LCST_IWCWORKTIMEINCOMSERVICE,WC_CAR01_LCST_IDOORCYCLCNT,WC_CAR03_LCST_IDOORCYCLCNT,WC_CAR05_LCST_IDOORCYCLCNT,WC_CAR07_LCST_IDOORCYCLCNT,WC_CAR01_LCST_IFLUSHCNT,WC_CAR03_LCST_IFLUSHCNT,WC_CAR05_LCST_IFLUSHCNT,WC_CAR07_LCST_IFLUSHCNT,WC_CAR01_LCST_IEJECTORCNT,WC_CAR03_LCST_IEJECTORCNT,WC_CAR05_LCST_IEJECTORCNT,WC_CAR07_LCST_IEJECTORCNT,WC_CAR01_LCST_IWATERTAPCNT,WC_CAR03_LCST_IWATERTAPCNT,WC_CAR05_LCST_IWATERTAPCNT,WC_CAR07_LCST_IWATERTAPCNT,WC_CAR01_LCST_IDELIMINGCNT,WC_CAR03_LCST_IDELIMINGCNT,WC_CAR05_LCST_IDELIMINGCNT,WC_CAR07_LCST_IDELIMINGCNT,WC_CAR01_LCST_IDEFROSTCNT,WC_CAR03_LCST_IDEFROSTCNT,WC_CAR05_LCST_IDEFROSTCNT,WC_CAR07_LCST_IDEFROSTCNT,WC_CAR01_LCST_IFWT05CNT,WC_CAR03_LCST_IFWT05CNT,WC_CAR05_LCST_IFWT05CNT,WC_CAR07_LCST_IFWT05CNT,WC_CAR01_LCST_IWWT95CNT,WC_CAR03_LCST_IWWT95CNT,WC_CAR05_LCST_IWWT95CNT,WC_CAR07_LCST_IWWT95CNT,WC_CAR01_LCST_FWCOUTOFSERVICE,WC_CAR03_LCST_FWCOUTOFSERVICE,WC_CAR05_LCST_FWCOUTOFSERVICE,WC_CAR07_LCST_FWCOUTOFSERVICE,WC_CAR01_LCST_IWCREADY,WC_CAR03_LCST_IWCREADY,WC_CAR05_LCST_IWCREADY,WC_CAR07_LCST_IWCREADY,WC_CAR01_LCST_FFWTEMPTY,WC_CAR03_LCST_FFWTEMPTY,WC_CAR05_LCST_FFWTEMPTY,WC_CAR07_LCST_FFWTEMPTY
count,2.319300e+04,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,23193.0,23193.000000,23193.000000,23193.000000,23193.000000,23193.0,23193.0,23193.0,23193.0,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,2.319300e+04,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,23193.000000,23193.0,23193.0,23193.0,23193.0,23193.0
mean,1.327080e+17,29784.833959,31221.024533,33736.410124,23047.295348,1013.243737,7117.591472,2095.547708,0.0,1224.069547,1223.543052,918.891735,2446.215669,0.0,0.0,0.0,0.0,1.342022e+08,1.414336e+08,1.373306e+08,1.726493e+08,2.529399e+09,2.733607e+09,2.502232e+09,3.276707e+09,1.860546e+06,2.691685e+06,1.723016e+06,3.799669e+06,2.065299e+07,1.183058e+07,1.260048e+07,2.681495e+07,2.829053e+06,9.120613e+06,3.996113e+06,1.954540e+07,1.089258e+06,1.036389e+06,6.917383e+05,1.466669e+06,367.780106,367.036606,368.155219,368.155219,119720.239814,128960.009486,112172.283663,161136.927823,3.486359e+07,8.220007e+08,9.700356e+07,6.989053e+07,1.719832e+07,1.121157e+08,4.015309e+07,8.969611e+06,536.182296,536.027465,0.011857,611.771526,75.627948,75.887811,611.542060,0.0,0.0,0.0,0.0,0.0
std,4.230343e+17,22085.354917,22281.728868,21221.427577,16365.415577,1614.828822,11280.366270,2846.441837,0.0,834.165990,833.529350,628.489666,1666.737358,0.0,0.0,0.0,0.0,3.894067e+07,4.104862e+07,3.984844e+07,5.009133e+07,2.194148e+09,2.356846e+09,2.167396e+09,2.838236e+09,1.425268e+06,1.808822e+06,1.368131e+06,2.572862e+06,1.572134e+07,7.929062e+06,8.531687e+06,1.815715e+07,2.314368e+06,7.088304e+06,3.778173e+06,1.487949e+07,7.360369e+05,6.946071e+05,5.134872e+05,9.931226e+05,248.522892